# Largest Lyapunov exponent calculation

### Algorithm

In [2]:
import numpy as np

In [3]:
# Logistic map
def logistic_map(n=2**15, r=4, x0=.4):
    x = np.zeros(n)
    x[0] = x0
    for i in range(n-1):
        x[i+1] = r*x[i]*(1-x[i])
    return x


# Henon map
def henon_map(n=1000000, a=1.4, b=0.3, x0=.4):
    x = np.zeros(n)
    x[0] = x0
    for i in range(1, len(x)):
        x[i] = 1 - a * x[i-1] ** 2 + b * x[i-1]
    return x


# Lorenz ts
def lorenz_ts(N=None):
    x = np.array([])
    i = 0
    with open("lorenz.txt") as f:
        for line in f:
            x = np.append(x, float(line))
            i += 1
            if N is not None and i == N:
                break
    return x

sine_data = np.sin(np.arange(0,1000,.01))

lorenz = lorenz_ts()
logistic = logistic_map()
henon = henon_map()

In [4]:
from sklearn.metrics.pairwise import euclidean_distances

def largest_exponent(series: np.array, J: int, m: int, t: float, 
                     mean_period: int = 1, trajectory_len: int = 20):
    # compute shape values
    N = series.shape[0]
    M = N - (m-1) * J

    # reconstruct with lag algorithm
    x = np.zeros((M, m))
    for i in range(M):
        indexes = np.ones(m) * i + np.arange(m) * J
        x[i] = series[indexes.astype(int)]
    
    # Find nearest neighbor
    distances = euclidean_distances(x)
    neighbors = np.argsort(distances[:,:M-trajectory_len], axis=1)[:,1]

    # mean rate of separation
    y = np.zeros(trajectory_len)
    for i in range(trajectory_len):
        neighbors_i = neighbors[:M-i] + i
        neighbors_i[neighbors_i >= M] = (M - 1)
        separation = distances[(np.arange(M - i) + i, neighbors_i)]
        separation = separation[neighbors[:M-i] + i < M]
        separation = separation[separation != 0]
        
        if separation.shape[0] == 0:
            y[i] = np.inf
        else:
            y[i] = np.log(separation).mean() / t

    y = y[np.isfinite(y)]
    slope, _ = np.polyfit(np.arange(1, len(y) + 1), y, 1)
    return slope

In [60]:
print(f"Logistic map {largest_exponent(logistic[:5000], J=1, m=2, t=1, trajectory_len=20):.3f}")

Logistic map 0.473


In [58]:
print(f"Lorenz {largest_exponent(lorenz[:5000], J=11, m=3, t=0.01, trajectory_len=130):.3f}")

Lorenz 1.732


In [39]:
print(f"Henon map {largest_exponent(henon[:5000], J=1, m=2, t=1, trajectory_len=20):.3f}")

Henon map -0.071


In [36]:

print(f"Sin {largest_exponent(sine_data[:5000], J=3, m=2, t=1, trajectory_len=30):.3f}")

Sin -0.000


In [7]:
from sklearn.metrics.pairwise import euclidean_distances

def largest_exponent_threshold(series: np.array, J: int, m: int, t: float = 0.1, threshold: float = 1):
    # compute shape values
    N = series.shape[0]
    M = N - (m-1) * J

    # reconstruct with lag algorithm
    x = np.zeros((M, m))
    for i in range(M):
        indexes = np.ones(m) * i + np.arange(m) * J
        x[i] = series[indexes.astype(int)]
    
    # Find nearest neighbor
    distances = euclidean_distances(x)
    neighbors = np.argsort(distances[:,:M-1], axis=1)[:,1]

    # mean rate of separation
    lambdas = []
    count = 0
    for i in range(M):
        k = 1
        j = neighbors[i]
        while np.linalg.norm(distances[i, j] - distances[i, j + k]) < threshold:
            k += 1
            if j + k >= M:
                k = -1
                break

        if k == -1 or k == 1:
            continue

        k -= 1
        count += 1
        lambdas.append((1/(t*k)) * np.log(distances[i, j + k] / distances[i, j]))

    return (1/count) * np.array(lambdas).sum()

In [23]:
print(f"Logistic map {largest_exponent_threshold(logistic[:5000], J=1, m=2, t=1, threshold=1):.3f}")

Logistic map 2.337


In [24]:
print(f"Lorenz {largest_exponent_threshold(lorenz[:5000], J=11, m=3, t=0.01, threshold=1):.3f}")

Lorenz 0.336


In [28]:
print(f"Henon map {largest_exponent_threshold(henon[:5000], J=1, m=2, t=1, threshold=3):.3f}")

ZeroDivisionError: division by zero

In [33]:
print(f"Sin {largest_exponent_threshold(sine_data[:5000], J=3, m=5, t=1, threshold=1):.3f}")

Sin 0.108
